# Telescope AWS Local Development

This notebook provides a comprehensive, step-by-step guide for manually running Terraform for AWS telescope testing. All commands should be run from the root of the repository and in a bash shell (Linux or WSL).

## Overview

This guide covers how to manually run Terraform for AWS. This interactive notebook version allows you to execute each step individually and see the results in real-time.

## Prerequisites

Before running this notebook, ensure you have the following tools installed:

* [Terraform - 1.7.3](https://developer.hashicorp.com/terraform/tutorials/azure-get-started/install-cli)
* [AWS CLI - 2.15.19](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux.html)
* [jq - 1.6-2.1ubuntu3](https://stedolan.github.io/jq/download/)
* [Kubectl - 1.31.0](https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/)
* [Helm - v3.16.1](https://helm.sh/docs/intro/install/)

### Install Python Packages for Jupyter (if not already installed)
```bash
pip install jupyter notebook ipykernel bash_kernel
python -m bash_kernel.install
```

You can install the Jupyter Extension from the VS Code marketplace: https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter

### Quick Start with Notebooks

To run this notebook locally:
```bash
jupyter notebook aws.ipynb
```
Once you run the command, go to http://localhost:8888/tree to access the Jupyter Notebook UI.

**Note**: All commands should be run from the root of the repository in a bash shell (Linux or WSL).

## 1. Prerequisites Check

First, let's verify that all required tools are installed and accessible.

In [ ]:
# Check prerequisites
echo "🔍 Checking prerequisites..."
echo "==========================================="

# Check Terraform
if command -v terraform &> /dev/null; then
    TERRAFORM_VERSION=$(terraform version -json | jq -r '.terraform_version')
    echo "✅ Terraform: $TERRAFORM_VERSION"
else
    echo "❌ Terraform not found"
fi

# Check AWS CLI
if command -v aws &> /dev/null; then
    AWS_VERSION=$(aws --version 2>&1)
    echo "✅ AWS CLI: $AWS_VERSION"
else
    echo "❌ AWS CLI not found"
fi

# Check jq
if command -v jq &> /dev/null; then
    JQ_VERSION=$(jq --version)
    echo "✅ jq: $JQ_VERSION"
else
    echo "❌ jq not found"
fi

# Check Kubectl
if command -v kubectl &> /dev/null; then
    KUBECTL_VERSION=$(kubectl version --client --output=yaml | grep gitVersion | awk '{print $2}')
    echo "✅ Kubectl: $KUBECTL_VERSION"
else
    echo "❌ Kubectl not found"
fi

# Check Helm
if command -v helm &> /dev/null; then
    HELM_VERSION=$(helm version --short)
    echo "✅ Helm: $HELM_VERSION"
else
    echo "❌ Helm not found"
fi

echo ""
echo "📍 Current directory: $(pwd)"
echo "💡 Make sure you're running from the telescope repository root"

# Navigate to repository root if not already there
if [ ! -f "scenarios" ] && [ ! -d "scenarios" ]; then
    echo "🔄 Navigating to repository root..."
    cd ../../../
    echo "📍 New directory: $(pwd)"
    export ROOT_DIR=$(pwd)
fi

## 2. Define Variables

Set environment variables for your specific test scenario. In this guide, we'll use `perf-eval/nap` scenario as the example and set the following variables:

**Note**: 
* `RUN_ID` should be a unique identifier since it is used to identify the resources based on tags as AWS has no concept of a resource group.
* `CREATION_TIME` is used to define the tags `creation_time` and `deletion_due_time`, which can be used to track provisioned resources in the AWS account. It must be within one hour of the time the terraform plan/apply commands are executed.
* These variables are not exhaustive and may vary depending on the scenario.

In [ ]:
# Define test scenario variables (from the repository root)
export SCENARIO_TYPE=perf-eval
export SCENARIO_NAME=nap
export RUN_ID=$(date +%s)
export CLOUD=aws
export REGION="us-east-2"
export K8S_MACHINE_TYPE=""
export K8S_OS_DISK_TYPE=""
export CREATION_TIME=$(date -uIseconds | sed 's/+00:00/Z/')
export TERRAFORM_MODULES_DIR=$ROOT_DIR/modules/terraform/$CLOUD
export TERRAFORM_INPUT_FILE=$ROOT_DIR/scenarios/$SCENARIO_TYPE/$SCENARIO_NAME/terraform-inputs/${CLOUD}.tfvars
export TERRAFORM_USER_DATA_PATH=$ROOT_DIR/scenarios/$SCENARIO_TYPE/$SCENARIO_NAME/scripts/user_data

echo "📋 Configuration Summary:"
echo "========================"
echo "Scenario Type: $SCENARIO_TYPE"
echo "Scenario Name: $SCENARIO_NAME"
echo "Run ID: $RUN_ID"
echo "Cloud: $CLOUD"
echo "Region: $REGION"
echo "Creation Time: $CREATION_TIME"
echo "Terraform Modules Dir: $TERRAFORM_MODULES_DIR"
echo "Terraform Input File: $TERRAFORM_INPUT_FILE"
echo "User Data Path: $TERRAFORM_USER_DATA_PATH"
echo ""
echo "⚠️  Note: RUN_ID should be unique as it's used to identify resources based on tags"
echo "⚠️  Note: CREATION_TIME must be within one hour of terraform plan/apply execution"

## 3. AWS Authentication

Configure and verify AWS credentials. You'll need to create access key and secret key for AWS CLI following the instructions [here](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_access-keys.html#Using_CreateAccessKey) if you don't have one yet.

### Login using the access key and secret key:

```bash
aws configure set aws_access_key_id <access-key>
aws configure set aws_secret_access_key <secret-access-key>
aws configure set region <test-region>
```

**Note**: Make sure you configure the region to be the same as where you want to provision the resources. Otherwise, you might get an error.

In [ ]:
# AWS authentication check
echo "🔐 AWS Authentication"
echo "====================="

# Check if AWS credentials are configured
if aws sts get-caller-identity &> /dev/null; then
    echo "✅ AWS credentials are configured"
    
    # Display current identity
    echo "📋 Current AWS Identity:"
    aws sts get-caller-identity --output table    
else
    echo "❌ AWS credentials not configured or invalid"
    echo "💡 Please run 'aws configure' to set up your credentials"
    echo "   export AWS_ACCESS_KEY_ID=your-access-key"
    echo "   export AWS_SECRET_ACCESS_KEY=your-secret-key"
    echo "   export AWS_DEFAULT_REGION=$REGION"
fi

# Set the region for this session
export AWS_DEFAULT_REGION=$REGION
echo ""
echo "🔧 Session region set to: $AWS_DEFAULT_REGION"

## 4. Prepare Terraform Input JSON

Create the JSON input for Terraform with all the configuration parameters.

In [ ]:
# Create INPUT_JSON variable for AWS
echo "📝 Preparing Terraform Input JSON"
echo "================================="

# Set INPUT_JSON variable as described in the README
# This variable is not exhaustive and may vary depending on the scenario
# For a full list of what can be set, look for `json_input` in file 
# `modules/terraform/aws/variables.tf` as the list will keep changing as we add more features
export INPUT_JSON=$(jq -n \
      --arg run_id $RUN_ID \
      --arg region $REGION \
      --arg creation_time $CREATION_TIME \
      --arg k8s_machine_type "$K8S_MACHINE_TYPE" \
      --arg k8s_os_disk_type "$K8S_OS_DISK_TYPE" \
      --arg user_data_path "$TERRAFORM_USER_DATA_PATH" \
      '{
      run_id: $run_id,
      region: $region,
      creation_time: $creation_time,
      k8s_machine_type: $k8s_machine_type,
      k8s_os_disk_type: $k8s_os_disk_type,
      user_data_path: $user_data_path,
      }' | jq 'with_entries(select(.value != null and .value != ""))')

echo "📋 Terraform Input JSON:"
echo "$INPUT_JSON" | jq .

echo ""
echo "📂 Terraform Input File: $TERRAFORM_INPUT_FILE"
if [ -f "$TERRAFORM_INPUT_FILE" ]; then
    echo "✅ Terraform input file exists"
    echo "📄 Contents of terraform input file:"
    cat "$TERRAFORM_INPUT_FILE"
else
    echo "⚠️  Terraform input file not found at: $TERRAFORM_INPUT_FILE"
fi

echo ""
echo "💡 Note: The jq command removes any null or empty values from the JSON object"
echo "💡 Note: Variables surrounded by double quotes are optional and can be removed if not needed"

## 5. Terraform Initialization & Plan

Initialize Terraform in the AWS modules directory.

In [ ]:
# Terraform initialization and plan
echo "🚀 Terraform Initialization & Plan"
echo "==================================="

# Change to terraform directory
pushd $TERRAFORM_MODULES_DIR

echo "📂 Current directory: $(pwd)"
echo "🔧 Initializing Terraform..."
terraform init

echo ""
echo "? Creating Terraform Plan..."
echo "============================="

# Update creation_time to current timestamp as per README instructions
export INPUT_JSON=$(echo $INPUT_JSON | jq -c '. + {creation_time: (now | todateiso8601)}')

echo "📅 Updated INPUT_JSON with current creation_time:"
echo "$INPUT_JSON" | jq .

echo ""
echo "🗺️  Running terraform plan..."
terraform plan \
  -var json_input="$INPUT_JSON" \
  -var-file $TERRAFORM_INPUT_FILE

echo ""
echo "✅ Terraform plan completed"
popd

## 6. Terraform Apply

Apply the Terraform configuration to provision AWS resources.

**⚠️ WARNING**: This will create actual AWS resources and may incur costs.

In [ ]:
# Terraform apply
echo "🚀 Terraform Apply"
echo "=================="

echo "⚠️  WARNING: This will provision actual AWS resources!"
echo "💰 This may incur costs in your AWS account"
echo "⏰ Resources will be tagged with creation time: $CREATION_TIME"
echo ""

pushd $TERRAFORM_MODULES_DIR

echo "🚀 Applying Terraform configuration..."
terraform apply \
  -var json_input="$INPUT_JSON" \
  -var-file $TERRAFORM_INPUT_FILE  --auto-approve

echo ""
echo "✅ Terraform apply completed"
popd

## 7. Cleanup Resources

**IMPORTANT**: Always clean up resources after testing to avoid unnecessary costs.

Cleanup test resources using terraform destroy.

In [ ]:
# Cleanup with Terraform
echo "🧹 Terraform Cleanup"
echo "====================="

echo "⚠️  WARNING: This will destroy all provisioned resources!"
echo "💾 Make sure to save any important data before proceeding"
echo ""

pushd $TERRAFORM_MODULES_DIR

echo "🗑️  Destroying Terraform resources..."
terraform destroy \
  -var json_input="$INPUT_JSON" \
  -var-file $TERRAFORM_INPUT_FILE --auto-approve

echo ""
echo "✅ Terraform destroy completed"
popd

## 8. References

For additional information and troubleshooting, refer to these resources:

* [Terraform AWS Provider](https://www.terraform.io/docs/providers/aws/index.html)
* [AWS CLI Documentation](https://docs.aws.amazon.com/cli/latest/)
* [AWS Console](https://aws.amazon.com/console/)
* [Terraform Documentation](https://developer.hashicorp.com/terraform/docs)
* [AWS IAM Access Keys Guide](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_access-keys.html#Using_CreateAccessKey)

## 9. Troubleshooting

### Common Issues:
1. **Region Mismatch**: Ensure AWS CLI region matches the region specified in variables
2. **Permissions**: Verify your AWS credentials have sufficient permissions for the resources being created
3. **Resource Limits**: Check AWS service limits if resource creation fails
4. **Timing**: Ensure `CREATION_TIME` is within one hour of terraform execution


The variable list changes as new features are added to the terraform modules.